In [1]:
import numpy as np
import cv2
import time
#import serial

In [2]:
# Defining 'VideoCapture' object
# and reading stream video from camera
camera = cv2.VideoCapture(0)
camera = cv2.VideoCapture(0)
#ser = serial.Serial()
#ser.baudrate = 9600
#ser.port = 'COM4'
#ser.open()
# Preparing variables for spatial dimensions of the frames
h, w = None, None

In [3]:
with open(r'D:\Education\Courses\ComputerVision\YOLO-3-OpenCV\CUSTOM_DATA_FOR_PROJECT_1\Trained_Weights\masknomask/obj.names') as f:
    # Getting labels reading every line
    # and putting them into the list
    labels = [line.strip() for line in f]
network = cv2.dnn.readNetFromDarknet(
    r'D:\Education\Courses\ComputerVision\YOLO-3-OpenCV\CUSTOM_DATA_FOR_PROJECT_1\Trained_Weights\masknomask/yolov4-tiny-obj2.cfg',
    r'D:\Education\Courses\ComputerVision\YOLO-3-OpenCV\CUSTOM_DATA_FOR_PROJECT_1\Trained_Weights\masknomask/yolov4-tiny-obj2_best.weights')

In [4]:
layers_names_all = network.getLayerNames()
layers_names_output = \
    [layers_names_all[i[0] - 1] for i in network.getUnconnectedOutLayers()]
probability_minimum = 0.4
threshold = 0.4
colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

In [5]:
while True:
    # Capturing frame-by-frame from camera
    _, frame = camera.read()

    # Getting spatial dimensions of the frame
    # we do it only once from the very beginning
    # all other frames have the same dimension
    if w is None or h is None:
        # Slicing from tuple only first two elements
        h, w = frame.shape[:2]
    # blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size, mean, swapRB=True)
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
                                 swapRB=True, crop=False)
    network.setInput(blob)  # setting blob as input to the network
    start = time.time()
    output_from_network = network.forward(layers_names_output)
    end = time.time()
    # Showing spent time for single current frame
    #print('Current frame took {:.5f} seconds'.format(end - start))
    # Preparing lists for detected bounding boxes,
    # obtained confidences and class's number
    bounding_boxes = []
    confidences = []
    class_numbers = []
    for result in output_from_network:
        # Going through all detections from current output layer
        for detected_objects in result:
            # Getting 80 classes' probabilities for current detected object
            scores = detected_objects[5:]
            # Getting index of the class with the maximum value of probability
            class_current = np.argmax(scores)
            # Getting value of probability for defined class
            confidence_current = scores[class_current]
            # Eliminating weak predictions with minimum probability
            if confidence_current > probability_minimum:   
                
                box_current = detected_objects[0:4] * np.array([w, h, w, h])

                # Now, from YOLO data format, we can get top left corner coordinates
                # that are x_min and y_min
                x_center, y_center, box_width, box_height = box_current
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))

                # Adding results into prepared lists
                bounding_boxes.append([x_min, y_min,
                                       int(box_width), int(box_height)])
                confidences.append(float(confidence_current))
                class_numbers.append(class_current)
                ######################################################
    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences,
                               probability_minimum, threshold) 
    if len(results) > 0:
        # Going through indexes of results
        for i in results.flatten():
            # Getting current bounding box coordinates,
            # its width and height
            x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
            box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]

            # Preparing colour for current bounding box
            # and converting from numpy array to list
            colour_box_current = colours[class_numbers[i]].tolist()

            # # # Check point
            # print(type(colour_box_current))  # <class 'list'>
            # print(colour_box_current)  # [172 , 10, 127]
            #frame[x_min:x_min + box_width,y_min:y_min + box_height] = cv2.GaussianBlur(frame[x_min:x_min + box_width,y_min:y_min + box_height], (21,21), cv2.BORDER_DEFAULT)
            # Drawing bounding box on the original current frame
            cv2.rectangle(frame, (x_min, y_min),
                          (x_min + box_width, y_min + box_height),
                          colour_box_current, 2)
            try:
                #frame[y_min:y_min + box_height,x_min:x_min + box_width] = cv2.GaussianBlur(frame[y_min:y_min + box_height,x_min:x_min + box_width], (51,51), cv2.BORDER_DEFAULT)
                frame[y_min:y_min + box_height,x_min:x_min + box_width] = cv2.blur(frame[y_min:y_min + box_height,x_min:x_min + box_width], (25,25))
                # Preparing text with label and confidence for current bounding box
                text_box_current = '{}: {:.4f}'.format(labels[int(class_numbers[i])],
                                                       confidences[i])
                #print("Object: ",text_box_current,x_min,y_min,box_height,box_width)
                # Putting text with label and confidence on the original image
                cv2.putText(frame, text_box_current, (x_min, y_min - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, colour_box_current, 4)
            except:
                text_box_current = '{}: {:.4f}'.format(labels[int(class_numbers[i])],
                                                   confidences[i])
                #print("Object: ",text_box_current,x_min,y_min,box_height,box_width)
                # Putting text with label and confidence on the original image
                cv2.putText(frame, text_box_current, (x_min, y_min - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, colour_box_current, 4)
    # Showing results obtained from camera in Real Time

    # Showing current frame with detected objects
    # Giving name to the window with current frame
    # And specifying that window is resizable
    cv2.namedWindow('Mask Detection YOLOv4Tiny', cv2.WINDOW_NORMAL)
    # Pay attention! 'cv2.imshow' takes images in BGR format
    cv2.imshow('Mask Detection YOLOv4Tiny', frame)

    # Breaking the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Releasing camera
camera.release()
# Destroying all opened OpenCV windows
cv2.destroyAllWindows()